# 0. Setting

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

main_directory = '/content/drive/MyDrive/Colab Notebooks/Foiegras'

if not os.path.exists(main_directory):
    os.makedirs(main_directory)
    print(f"'{main_directory}' 디렉토리를 생성했습니다.")
else:
    print(f"'{main_directory}' 디렉토리가 이미 존재합니다.")

'/content/drive/MyDrive/Colab Notebooks/Foiegras' 디렉토리가 이미 존재합니다.


In [ ]:
os.chdir(main_directory)
print(f"'{os.getcwd()}' 디렉토리에서 작업 중입니다.")

'/content/drive/MyDrive/Colab Notebooks/Foiegras' 디렉토리에서 작업 중입니다.


# 1. ETF Info

## 1) Loop

- 164개 상품, etf_df에 정리

In [ ]:
!pip install llama-index-readers-file pymupdf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.2 MB/s eta 0:00:00


In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import PyMuPDFReader
from collections import defaultdict
import base64
import json
import http.client
import time
import pandas as pd
from tqdm import tqdm

In [ ]:
def split_text_by_keywords(text_list, keywords):
    result = defaultdict(list)
    current_keyword = None

    for item in text_list:
        if item in keywords:
            current_keyword = item
        elif current_keyword:
            result[current_keyword].append(item)

    if '누적성과그래프' in result:
        del result['누적성과그래프']

    return dict(result)

class ClovaEmbedding:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/embedding/v2/bbdd4fe4d10e4114aa67715014ecf1af', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['embedding']
        else:
            return 'Error'

In [ ]:
useless_parts = ['* 포트폴리오정보는PDF 구성내역에기초한것으로실제보유내역과다를수있습니다.',
                 '* 상기업종비중은GICS 대분류기준입니다.',
                 '* 위의수익률정보는배당금재투자를가정하였음.', '단, 아래그래프는배당금제외한NAV 기준임.',
                 '변동성: 최근1년간일간수익률기준표준편차연환산(운용기간1년이내인경우전체운용일수기준)',
                 '* 위의수익률정보및아래그래프는운용기간중발생한분배금재투자를가정하였음.',
                 '※ ETF는실적배당상품으로운용결과에따른이익또는손실이투자자에게귀속됩니다.',
                 '※ 예금자보호법에따라예금보험공사가보호하지', '않으며, 원본의손실이발생할수있습니다.',
                 '※ ETF를취득하기전에(투자대상, 환매방법및보수등에관하여) 투자설명서를반드시읽어', '주시기바랍니다.',
                 '※ 과거의운용실적이미래의운용수익을보장하지않습니다.',
                 '(www.tigeretf.com)']
keywords = ['기초지수', '분배현황(최근2년)', '주요보유현황', '업종비중', '상위10종목', '기본정보', '거래정보', '운용성과(%)', '누적성과그래프', '투자포인트']

In [ ]:
clova_embedding = ClovaEmbedding(
    host='clovastudio.apigw.ntruss.com',
    api_key='MASKED',
    api_key_primary_val = 'MASKED',
    request_id='MASKED'
    )

fact_names = os.listdir(os.path.join(main_directory, "dataset_2/etf_factsheet"))
fact_names.sort()
pros_names = os.listdir(os.path.join(main_directory, "dataset_2/etf_prospectus"))
pros_names.sort()
tiger_df = pd.read_excel(os.path.join(main_directory, 'dataset_2/etf_list.xlsx'), dtype=str)

etf_df = pd.DataFrame(columns=['ticker',
                               'name',
                               'risk', # 투자위험등급
                               'private', # 개인연금 가능
                               'retire', # 퇴직연금 가능
                               'date', # 운용보고서 기준일
                               'sent1', 'sent1_emb', # 기초지수 (벤치마크)
                               'sent2', 'sent2_emb', # 분배금 지급 현황
                               'sent3', 'sent3_emb', # 자산구성 (포트폴리오)
                               'sent4', 'sent4_emb', # 기본정보
                               'sent5', 'sent5_emb', # 거래정보
                               'sent6', 'sent6_emb', # 운용성과
                               'sent7', 'sent7_emb']) # 투자포인트

In [ ]:
for fact_name, pros_name in tqdm(zip(fact_names, pros_names), desc='Processing ETF Info', total=len(fact_names)):

    # 1

    fact_path = os.path.join(main_directory, "dataset_2/etf_factsheet", fact_name)
    loader = PyMuPDFReader()
    document = loader.load_data(file_path=fact_path, metadata=True)
    text = document[0].to_dict()['text']

    for useless_part in useless_parts:
        text = text.replace(useless_part, '')

    # 2

    ticker, _, date = fact_path.split('/')[-1].split('.')[0].split('_')

    # 3

    text_list = text.split('\n')
    split_texts = split_text_by_keywords(text_list, keywords)

    # 4

    sent_1 = ' '.join(['[기초지수]', '_'.join(split_texts['기초지수'])])
    sent_1_emb = clova_embedding.execute({"text": sent_1})

    sent_2 = ' '.join(['[분배현황(최근2년)]', '_'.join(split_texts['분배현황(최근2년)'])])
    sent_2_emb = clova_embedding.execute({"text": sent_2})

    sent_3a, sent_3b, sent_3c = 'None', 'None', 'None'
    try:
        sent_3a = '_'.join(split_texts['주요보유현황'])
    except KeyError: pass
    try:
        sent_3b = '_'.join(split_texts['업종비중'])
    except KeyError: pass
    try:
        sent_3c = '_'.join(split_texts['상위10종목'])
    except KeyError: pass
    sent_3 = ' '.join(['[주요보유현황]', sent_3a, '[업종비중]', sent_3b, '[상위10종목]', sent_3c])
    sent_3_emb = clova_embedding.execute({"text": sent_3})

    sent_4 = ' '.join(['[기본정보]', '_'.join(split_texts['기본정보'])])
    sent_4_emb = clova_embedding.execute({"text": sent_4})

    sent_5 = ' '.join(['[거래정보]', '_'.join(split_texts['거래정보'])])
    sent_5_emb = clova_embedding.execute({"text": sent_5})

    sent_6 = ' '.join(['[운용성과(%)]', '_'.join(split_texts['운용성과(%)'])])
    sent_6_emb = clova_embedding.execute({"text": sent_6})

    sent_7 = ' '.join(['[투자포인트]', '_'.join(split_texts['투자포인트'])])
    sent_7_emb = clova_embedding.execute({"text": sent_7})

    time.sleep(20) # API 호출 제한 (Tokens per Minute)

    # 5

    pros_path = os.path.join(main_directory, "dataset_2/etf_prospectus", pros_name)
    loader = PyMuPDFReader()
    document = loader.load_data(file_path=pros_path, metadata=True)

    risk_level = document[0].to_dict()['text'].split('투자위험등급 ')[-1].split('\n')[0]

    # 6

    item_name = tiger_df[tiger_df['종목코드'] == ticker]['ETF명'].unique()[0]
    private_allowed = (tiger_df[tiger_df['종목코드'] == ticker]['개인연금'] != '0%').unique()[0]
    retire_allowed = (tiger_df[tiger_df['종목코드'] == ticker]['퇴직연금'] != '0%').unique()[0]

    # 7

    etf_df.loc[len(etf_df)] = [ticker,
                               item_name,
                               risk_level,
                               private_allowed,
                               retire_allowed,
                               date,
                               sent_1, sent_1_emb,
                               sent_2, sent_2_emb,
                               sent_3, sent_3_emb,
                               sent_4, sent_4_emb,
                               sent_5, sent_5_emb,
                               sent_6, sent_6_emb,
                               sent_7, sent_7_emb]

Processing ETF Info: 100%|██████████| 164/164 [1:11:54<00:00, 26.31s/it]


## 2) DB

- csv로 저장, 불러오기

### Save

In [ ]:
import pandas as pd

etf_path = os.path.join(main_directory, "dataset_2", "etf_df.csv")

etf_df.to_csv(etf_path, index=False)
print("Saved to CSV")

Saved to CSV


### Fix

- 텍스트 잘못 긁은 것, 수작업으로 파악, 새로 embedding

In [ ]:
temp = pd.read_excel(os.path.join(main_directory, 'dataset_2/etf_df.xlsx'), dtype=str)

In [ ]:
err_1 = temp[temp['ticker'] == '456610']['sent3'].values[0]
print(err_1)
fix_1 = clova_embedding.execute({"text": err_1})
temp.loc[temp['ticker'] == '456610', 'sent3_emb'] = [fix_1]

[주요보유현황] 종목명_만기일_편입비중(%)_Solactive SOFR TRS 2__95.10_Solactive SOFR TRS 1__1.55_Solactive SOFR Index TRS 231130-04__1.54_[USD] 예금__1.18_ [업종비중] None [상위10종목] None


In [ ]:
err_2 = temp[temp['ticker'] == '456610']['sent7'].values[0]
print(err_2)
fix_2 = clova_embedding.execute({"text": err_2})
temp.loc[temp['ticker'] == '456610', 'sent7_emb'] = [fix_2]

[투자포인트] 1. 공신력있는미국달러무위험지표금리SOFR에투자_2. 원달러환노출상품으로유휴USD자금운용가능_ _ _ _


In [ ]:
err_3 = temp[temp['ticker'] == '476550']['sent3'].values[0]
print(err_3)
fix_3 = clova_embedding.execute({"text": err_3})
temp.loc[temp['ticker'] == '476550', 'sent3_emb'] = [fix_3]

[주요보유현황] 종목명_편입비중(%)_iShares 20+ Year Treasury Bond ETF_24.89_SPDR Portfolio Long Term Treasury ETF_19.92_PIMCO 25+ Year Zero Coupon U.S. Treasury Index Exchange-Traded Fund_15.40_Vanguard Long-Term Treasury ETF_14.94_Vanguard Extended Duration Treasury ETF_9.93_Schwab Long-Term U.S. Treasury ETF_4.98_원화예금_9.99 [업종비중] None [상위10종목] None


In [ ]:
err_4 = temp[temp['ticker'] == '476550']['sent7'].values[0]
print(err_4)
fix_4 = clova_embedding.execute({"text": err_4})
temp.loc[temp['ticker'] == '476550', 'sent7_emb'] = [fix_4]

[투자포인트] 1. 금리인하를기대하나인내가필요할때적합_2. 자본차익과월배당을동시에_3. TIGER 월배당자산배분솔루션_ _ _ _


In [ ]:
err_5 = temp['sent6'].values
for idx, err in tqdm(enumerate(err_5), total=len(err_5)):
    fix = clova_embedding.execute({"text": err})
    temp.loc[idx, 'sent6_emb'] = fix
    time.sleep(1)

100%|██████████| 164/164 [04:55<00:00,  1.80s/it]


In [ ]:
def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(item)
        else:
            flat_list.append(item)
    return flat_list

temp.loc[temp['ticker'] == '456610', 'sent3_emb'] = temp.loc[temp['ticker'] == '456610', 'sent3_emb'].apply(flatten_list)
temp.loc[temp['ticker'] == '456610', 'sent7_emb'] = temp.loc[temp['ticker'] == '456610', 'sent7_emb'].apply(flatten_list)
temp.loc[temp['ticker'] == '476550', 'sent3_emb'] = temp.loc[temp['ticker'] == '476550', 'sent3_emb'].apply(flatten_list)
temp.loc[temp['ticker'] == '476550', 'sent7_emb'] = temp.loc[temp['ticker'] == '476550', 'sent7_emb'].apply(flatten_list)
temp['sent6_emb'] = temp['sent6_emb'].apply(flatten_list)

In [ ]:
temp

,ticker,name,risk,private,retire,date,sent1,sent1_emb,sent2,sent2_emb,sent3,sent3_emb,sent4,sent4_emb,sent5,sent5_emb,sent6,sent6_emb,sent7,sent7_emb
0,091220,TIGER 은행,2등급[높은 위험],True,True,240628,[기초지수] KRX BANKS 지수는유가증권시장과코스닥시장에상장된_은행주10종목으로...,"[0.42919922, 0.08654785, -0.83691406, 0.402587...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.7949219, 0.5932617, 0.12841797, -0.4663086...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_금융_98.39 [...,"[-1.0429688, 0.22717285, -0.9248047, -0.708007...","[기본정보] 최초설정일_2006년6월26일_기초자산_국내주식_순자산총액_9,389백...","[-0.8251953, 0.00013148785, -0.33447266, 0.145...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_091220_블룸버그티커(펀드)...,"[-1.5058594, -0.6303711, -1.5068359, 0.4494629...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-0.9609375, -0.00920105, -1.0136719, -0.31640...",[투자포인트] 1. 국내상장은행주에투자_2. 금리인상시수혜기대_ _ _ _,"[0.7832031, 0.36669922, -1.3603516, 0.8881836,..."
1,091230,TIGER 반도체,1등급[매우 높은 위험],True,True,240628,[기초지수] KRX SEMICON 지수는전기·전자산업에속한반도체칩_제조기업과다른산업...,"[-0.9189453, -0.19555664, 0.46069336, -0.50048...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.5615234, 0.72753906, -0.10803223, -0.29663...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_98.92...,"[-1.515625, 0.6611328, -0.76660156, 0.00830078...","[기본정보] 최초설정일_2006년6월26일_기초자산_국내주식_순자산총액_237,65...","[-0.80859375, 0.035705566, -0.28686523, 0.0960...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_091230_블룸버그티커(펀드)...,"[-1.6787109, -0.8769531, -1.3085938, 0.3642578...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-1.0839844, -0.10076904, -0.8847656, -0.60156...","[투자포인트] 1. 4차산업혁명의핵심밸류체인, 반도체산업_2. 국내반도체밸류체인을대...","[0.36816406, 1.1679688, -0.8496094, 0.7319336,..."
2,098560,TIGER 방송통신,3등급[다소 높은 위험],True,True,240628,[기초지수] KRX 방송통신지수는유가증권시장과코스닥시장에서거래되고_있는보통주를대상으...,"[-1.2070312, 0.026062012, -0.042633057, -0.192...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.7529297, 0.578125, 0.08703613, -0.37280273...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_커뮤니케이션서비스_...,"[-1.7070312, 1.0595703, -0.68310547, -1.125976...","[기본정보] 최초설정일_2007년9월6일_기초자산_국내주식_순자산총액_3,113백만...","[-0.9824219, 0.12927246, -0.28149414, 0.113769...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_098560_블룸버그티커(펀드)...,"[-1.8505859, -0.8823242, -0.9321289, 0.2768554...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-1.0791016, -0.07159424, -0.7705078, -1.09375...","[투자포인트] 1. 방송통신기업에분산투자하는ETF_2. 5G, OTT 등신성장동력에...","[0.20544434, -0.46728516, -0.3100586, 0.554199..."
3,102110,TIGER 200,2등급[높은 위험],True,True,240628,[기초지수] KOSPI200 지수는한국거래소유가증권시장에상장된종목중_유동주식수가중한...,"[-0.63916016, -0.2956543, -0.73583984, 0.50683...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.7978516, 0.6040039, 0.0158844, -0.46166992...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_45.58...,"[-1.2207031, 0.07513428, -0.6801758, 0.0046348...","[기본정보] 최초설정일_2008년4월2일_기초자산_KOSPI200_순자산총액_2,2...","[-1.0917969, -0.07946777, -0.54345703, 0.48339...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_102110_블룸버그티커(펀드)...,"[-2.1113281, -0.68310547, -1.2695312, 0.647949...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-1.3789062, -0.22961426, -1.1748047, -0.45312...",[투자포인트] 1. 한국대표시장지수인KOSPI 200 지수수익률추종_2. 국내최저수...,"[-0.95166016, 0.2824707, -1.2734375, 0.3269043..."
4,105010,TIGER 라틴35,2등급[높은 위험],True,True,240628,[기초지수] BNY Latin America 35 ADR지수는미국시장에상장된_라틴아...,"[0.34155273, -1.2558594, 0.2932129, 0.18579102...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.6376953, 0.57714844, -0.0110321045, -0.502...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_소재_23.00_2...,"[-0.30151367, 0.23669434, -0.7734375, -0.34765...","[기본정보] 최초설정일_2008년8월26일_기초자산_증권예탁증서_순자산총액_5,53...","[-0.95458984, -0.14709473, -0.24731445, 0.2983...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_105010_블룸버그티커(펀드)...,"[-1.8535156, -1.0332031, -1.3505859, 0.3212890...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-1.3740234, -0.34399414, -1.2832031, -0.51220...","[투자포인트] 1. 중남미지역대형주35종목에분산투자_2. 브라질, 멕시코, 칠레등남...","[0.09686279, -0.07543945, -0.49414062, 0.92333..."
...,...,...,...,...,...,...,..

In [ ]:
temp.to_excel(os.path.join(main_directory, 'dataset_2/etf_df_fixed.xlsx'), index=False)

### Load

In [ ]:
import pandas as pd
import ast

etf_df_fixed = pd.read_excel(os.path.join(main_directory, 'dataset_2/etf_df_fixed.xlsx'), dtype=str)

# str to list

emb_cols = ['sent1_emb', 'sent2_emb', 'sent3_emb', 'sent4_emb', 'sent5_emb', 'sent6_emb', 'sent7_emb']

for i in range(1, 8):
    emb_col = f'sent{i}_emb'
    etf_df_fixed[emb_col] = etf_df_fixed[emb_col].apply(ast.literal_eval)

# risk (int)

def extract_risk_level(risk_str):
    return int(risk_str.split('등급')[0])

etf_df_fixed['risk'] = etf_df_fixed['risk'].str.strip().apply(extract_risk_level)

In [ ]:
etf_df_fixed

,ticker,name,risk,private,retire,date,sent1,sent1_emb,sent2,sent2_emb,sent3,sent3_emb,sent4,sent4_emb,sent5,sent5_emb,sent6,sent6_emb,sent7,sent7_emb
0,091220,TIGER 은행,2,True,True,240628,[기초지수] KRX BANKS 지수는유가증권시장과코스닥시장에상장된_은행주10종목으로...,"[0.42919922, 0.08654785, -0.83691406, 0.402587...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.7949219, 0.5932617, 0.12841797, -0.4663086...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_금융_98.39 [...,"[-1.0429688, 0.22717285, -0.9248047, -0.708007...","[기본정보] 최초설정일_2006년6월26일_기초자산_국내주식_순자산총액_9,389백...","[-0.8251953, 0.00013148785, -0.33447266, 0.145...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_091220_블룸버그티커(펀드)...,"[-1.5058594, -0.6303711, -1.5068359, 0.4494629...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-0.9609375, -0.00920105, -1.0136719, -0.31640...",[투자포인트] 1. 국내상장은행주에투자_2. 금리인상시수혜기대_ _ _ _,"[0.7832031, 0.36669922, -1.3603516, 0.8881836,..."
1,091230,TIGER 반도체,1,True,True,240628,[기초지수] KRX SEMICON 지수는전기·전자산업에속한반도체칩_제조기업과다른산업...,"[-0.9189453, -0.19555664, 0.46069336, -0.50048...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.5615234, 0.72753906, -0.10803223, -0.29663...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_98.92...,"[-1.515625, 0.6611328, -0.76660156, 0.00830078...","[기본정보] 최초설정일_2006년6월26일_기초자산_국내주식_순자산총액_237,65...","[-0.80859375, 0.035705566, -0.28686523, 0.0960...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_091230_블룸버그티커(펀드)...,"[-1.6787109, -0.8769531, -1.3085938, 0.3642578...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-1.0839844, -0.10076904, -0.8847656, -0.60156...","[투자포인트] 1. 4차산업혁명의핵심밸류체인, 반도체산업_2. 국내반도체밸류체인을대...","[0.36816406, 1.1679688, -0.8496094, 0.7319336,..."
2,098560,TIGER 방송통신,3,True,True,240628,[기초지수] KRX 방송통신지수는유가증권시장과코스닥시장에서거래되고_있는보통주를대상으...,"[-1.2070312, 0.026062012, -0.042633057, -0.192...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.7529297, 0.578125, 0.08703613, -0.37280273...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_커뮤니케이션서비스_...,"[-1.7070312, 1.0595703, -0.68310547, -1.125976...","[기본정보] 최초설정일_2007년9월6일_기초자산_국내주식_순자산총액_3,113백만...","[-0.9824219, 0.12927246, -0.28149414, 0.113769...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_098560_블룸버그티커(펀드)...,"[-1.8505859, -0.8823242, -0.9321289, 0.2768554...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-1.0791016, -0.07159424, -0.7705078, -1.09375...","[투자포인트] 1. 방송통신기업에분산투자하는ETF_2. 5G, OTT 등신성장동력에...","[0.20544434, -0.46728516, -0.3100586, 0.554199..."
3,102110,TIGER 200,2,True,True,240628,[기초지수] KOSPI200 지수는한국거래소유가증권시장에상장된종목중_유동주식수가중한...,"[-0.63916016, -0.2956543, -0.73583984, 0.50683...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.7978516, 0.6040039, 0.0158844, -0.46166992...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_정보기술_45.58...,"[-1.2207031, 0.07513428, -0.6801758, 0.0046348...","[기본정보] 최초설정일_2008년4월2일_기초자산_KOSPI200_순자산총액_2,2...","[-1.0917969, -0.07946777, -0.54345703, 0.48339...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_102110_블룸버그티커(펀드)...,"[-2.1113281, -0.68310547, -1.2695312, 0.647949...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-1.3789062, -0.22961426, -1.1748047, -0.45312...",[투자포인트] 1. 한국대표시장지수인KOSPI 200 지수수익률추종_2. 국내최저수...,"[-0.95166016, 0.2824707, -1.2734375, 0.3269043..."
4,105010,TIGER 라틴35,2,True,True,240628,[기초지수] BNY Latin America 35 ADR지수는미국시장에상장된_라틴아...,"[0.34155273, -1.2558594, 0.2932129, 0.18579102...",[분배현황(최근2년)] 분배금지급일_주당분배율(%)_주당분배금(원)_2024.05....,"[-1.6376953, 0.57714844, -0.0110321045, -0.502...",[주요보유현황] None [업종비중] No._업종_비중(%)_1_소재_23.00_2...,"[-0.30151367, 0.23669434, -0.7734375, -0.34765...","[기본정보] 최초설정일_2008년8월26일_기초자산_증권예탁증서_순자산총액_5,53...","[-0.95458984, -0.14709473, -0.24731445, 0.2983...",[거래정보] 상장거래소_KRX 유가증권시장_상장코드_105010_블룸버그티커(펀드)...,"[-1.8535156, -1.0332031, -1.3505859, 0.3212890...",[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,"[-1.3740234, -0.34399414, -1.2832031, -0.51220...","[투자포인트] 1. 중남미지역대형주35종목에분산투자_2. 브라질, 멕시코, 칠레등남...","[0.09686279, -0.07543945, -0.49414062, 0.92333..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# 2. User Info

## 1) Option

In [ ]:
user_risk_level = 2 # 투자성향 5단계
user_private_pension = False # 개인연금
user_retire_pension = True # 퇴직연금

## 2) Text Query

In [ ]:
user_info = "나는 주식에 대해서 잘 몰라. 그래서 미국 50대 기업에 자동으로 투자하고, 돈을 불려주는 상품을 추천받고 싶어"

# 3. RAG

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


def find_top_k_sentences(user_info, etf_df_fixed, k=10):
    user_emb = clova_embedding.execute({"text": user_info})
    user_emb = np.array(user_emb).reshape(1, -1)
    similarities = []

    for idx, row in etf_df_fixed.iterrows():
        for i in range(1, 8):
            sent_emb = row[f'sent{i}_emb']
            sent_emb = np.array(sent_emb).reshape(1, -1)
            sim = cosine_similarity(user_emb, sent_emb)[0][0]
            similarities.append((sim, row[f'sent{i}'], row['ticker'], row['name']))

    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)[:k]
    similarities_df = pd.DataFrame(similarities, columns=['similarity', 'sentence', 'ticker', 'name'])
    return similarities_df

## 1) Candidates

- Text Query와 유사한 Top-K ETF 선정.

In [ ]:
candidates_df = find_top_k_sentences(user_info, etf_df_fixed, k=10)
candidates_df

,similarity,sentence,ticker,name
0,0.599564,[투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은...,A472170,TIGER미국테크TOP10채권혼합
1,0.585364,[투자포인트] 1. 미국시장을대표하는테크TOP10 기업에투자_2. 옵션프리미엄으로성...,A474220,TIGER미국테크TOP10+10%프리미엄
2,0.584069,[기초지수] Bloomberg U.S. Tech Top10 + 10% Premium...,A474220,TIGER미국테크TOP10+10%프리미엄
3,0.578601,[투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은...,A472160,TIGER 미국테크TOP10INDXX(H)
4,0.571596,[투자포인트] 1. 성장하는미국시장과동반상승하는S&P500 지수에투자_2. 분배금자...,448290,TIGER 미국S&P500TR(H)
5,0.568902,[투자포인트] 1. 글로벌선도기업중심으로장기우상향이기대되는나스닥100 지_수에투자_...,441680,TIGER 미국나스닥100커버드콜(합성)
6,0.568011,"[투자포인트] 1. 자율주행, AI, 반도체로봇등을개발하며전기차시장을선도하는_테슬라...",447770,TIGER 테슬라채권혼합Fn
7,0.567657,[투자포인트] 1. 미국주식시장을대표하는500개기업에분산투자_2. 미국주식시장을하나...,360750,TIGER 미국S&P500
8,0.566047,[투자포인트] 1. 글로벌기술선도대표100대기업에투자_2. 분배금자동재투자로분배금과...,448300,TIGER 미국나스닥100TR(H)
9,0.557054,[투자포인트] 1. 미국대표성장기업들에레버리지로투자_2. 해외상장대비편리한매매_ _...,418660,TIGER 미국나스닥100레버리지(합성)


## 2) Filter

- 투자성향, 개인/퇴직연금 구분 등 조건에 맞지 않는 ETF는 제외.

In [ ]:
candidates = etf_df_fixed[etf_df_fixed['ticker'].isin(candidates_df['ticker'].unique())]

risk_dict = {1: [1, 2, 3, 4, 5, 6],
             2: [2, 3, 4, 5, 6],
             3: [4, 5, 6],
             4: [5, 6],
             5: [6]}

candidates = candidates[candidates['risk'].isin(risk_dict[user_risk_level])]

if user_private_pension:
    candidates = candidates[candidates['private'] == 'True']
if user_retire_pension:
    candidates = candidates[candidates['retire'] == 'True']

filtered_tickers = candidates['ticker'].values
filtered_tickers

array(['195930', '245340', '277640', '360750', '381170', '429010',
       '441680', '447770', '448290', '448300', '453870', '482730',
       'A472160', 'A472170', 'A474220'], dtype=object)

In [ ]:
selected_df = candidates_df[candidates_df['ticker'].isin(filtered_tickers)]
selected_df

,similarity,sentence,ticker,name
0,0.599564,[투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은...,A472170,TIGER미국테크TOP10채권혼합
1,0.585364,[투자포인트] 1. 미국시장을대표하는테크TOP10 기업에투자_2. 옵션프리미엄으로성...,A474220,TIGER미국테크TOP10+10%프리미엄
2,0.584069,[기초지수] Bloomberg U.S. Tech Top10 + 10% Premium...,A474220,TIGER미국테크TOP10+10%프리미엄
3,0.578601,[투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은...,A472160,TIGER 미국테크TOP10INDXX(H)
4,0.571596,[투자포인트] 1. 성장하는미국시장과동반상승하는S&P500 지수에투자_2. 분배금자...,448290,TIGER 미국S&P500TR(H)
5,0.568902,[투자포인트] 1. 글로벌선도기업중심으로장기우상향이기대되는나스닥100 지_수에투자_...,441680,TIGER 미국나스닥100커버드콜(합성)
6,0.568011,"[투자포인트] 1. 자율주행, AI, 반도체로봇등을개발하며전기차시장을선도하는_테슬라...",447770,TIGER 테슬라채권혼합Fn
7,0.567657,[투자포인트] 1. 미국주식시장을대표하는500개기업에분산투자_2. 미국주식시장을하나...,360750,TIGER 미국S&P500
8,0.566047,[투자포인트] 1. 글로벌기술선도대표100대기업에투자_2. 분배금자동재투자로분배금과...,448300,TIGER 미국나스닥100TR(H)
10,0.554620,[기초지수] Nasdaq Next Generation 100 지수는나스닥100지수에...,429010,TIGER 미국나스닥넥스트100


## 3) Generate Answer

- 골라진 ETF texts + Text query 를 ClovaChat 에 넣어 답변 생성

In [ ]:
import requests

class ClovaChat:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        result_data = None

        with requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8")
                    print(decoded_line)
                    if 'seed' in decoded_line:
                        result_data = decoded_line

        return result_data

clova_chat = ClovaChat(
    host='https://clovastudio.stream.ntruss.com',
    api_key='MASKED',
    api_key_primary_val='MASKED',
    request_id='MASKED'
)

In [ ]:
selected_sentences = []

for idx, row in selected_df.iterrows():
    sent = f"<BOS> {row['ticker']}: {row['sentence']}"
    selected_sentences.append(sent)

texts = ' <EOS> '.join(selected_sentences)
texts

'<BOS> A472170: [투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은미국빅테크_3. 퇴직연금내100%까지투자가능한ETF_ <EOS> <BOS> A474220: [투자포인트] 1. 미국시장을대표하는테크TOP10 기업에투자_2. 옵션프리미엄으로성장주에월배당을더하다_3. TIGER +% 프리미엄격주배당솔루션_ _ _ _ <EOS> <BOS> A474220: [기초지수] Bloomberg U.S. Tech Top10 + 10% Premium Covered_Call 지수나스닥에상장된미국의대형기술주10종목에_투자하며동시에옵션을활용하여연간옵션프리미엄수익_10%를목표로하여매월옵션을일부매도하는전략지수 <EOS> <BOS> A472160: [투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은미국빅테크_3. 미국의성장을가장잘반영하는新대표지수_ _ _ _ <EOS> <BOS> 448290: [투자포인트] 1. 성장하는미국시장과동반상승하는S&P500 지수에투자_2. 분배금자동재투자로분배금과세이연혜택_3. 퇴직연금으로도활용가능한환헤지전략제공_ _ _ _ <EOS> <BOS> 441680: [투자포인트] 1. 글로벌선도기업중심으로장기우상향이기대되는나스닥100 지_수에투자_2. 매달옵션프리미엄을통해꾸준한현금흐름창출_3. 월분배금을비롯한인컴의복리효과로장기투자시수익률안정화_ _ _ _ <EOS> <BOS> 447770: [투자포인트] 1. 자율주행, AI, 반도체로봇등을개발하며전기차시장을선도하는_테슬라에투자_2. 퇴직연금내100%까지투자가능한ETF_ _ _ _ <EOS> <BOS> 360750: [투자포인트] 1. 미국주식시장을대표하는500개기업에분산투자_2. 미국주식시장을하나의ETF로추종_ _ _ _ <EOS> <BOS> 448300: [투자포인트] 1. 글로벌기술선도대표100대기업에투자_2. 분배금자동재투자로분배금과세이연혜택_3. 퇴직연금으로도활용가능한환헤지전략제공_ _ _ _ <EOS> <BOS> 429010: [

In [ ]:
preset_text = [{"role":"system",
                "content": ' '.join(["당신은 증권 전문가입니다.",
                                     "고객은 상장지수펀드 ETF에 관한 질문을 하고, 이는 <Query> 로 입력받습니다.",
                                     "ETF 정보를 담은 데이터베이스를 <DB> 로 입력받습니다.",
                                     "데이터베이스는 운용보고서를 발췌한 것으로, '<BOS> Ticker: [카테고리] 텍스트 <EOS>' 형태 문장으로 이뤄져있습니다."
                                     "반드시 데이터베이스 정보에 근거해 답변하십시오.",
                                     "답변할 때 '데이터베이스' 라는 단어는 사용하지 마십시오.",
                                     "데이터베이스에 없는 정보에 대해서는 '죄송합니다. 그 질문은 답변드릴 수 없습니다.' 라고만 답하십시오."])},
                  {"role":"user","content":f"<Query>: {user_info} </Query> <DB> {texts} </DB>"}]

request_data = {
    'messages': preset_text,
    'topP': 0.8,
    'topK': 0,
    'maxTokens': 2048,
    'temperature': 0.5, 'repeatPenalty': 5.0,
    'stopBefore': [],
    'includeAiFilters': True,
    'seed': 42
}

output = clova_chat.execute(request_data)

id:68009dfe-6d00-46fb-9a42-13ef7a39a07a
event:token
data:{"message":{"role":"assistant","content":"미국"},"index":0,"inputLength":1265,"outputLength":1,"stopReason":null}
id:616bb439-27ba-4d1e-b361-4967485993ad
event:token
data:{"message":{"role":"assistant","content":" 50"},"index":0,"inputLength":1265,"outputLength":1,"stopReason":null}
id:759186c1-5ee7-485e-90bf-efec4101c117
event:token
data:{"message":{"role":"assistant","content":"대"},"index":0,"inputLength":1265,"outputLength":1,"stopReason":null}
id:6f4cf9b1-343b-4c8c-968d-e32a89591170
event:token
data:{"message":{"role":"assistant","content":" 기업"},"index":0,"inputLength":1265,"outputLength":1,"stopReason":null}
id:687a1610-5c13-45ac-a246-989391bc482a
event:token
data:{"message":{"role":"assistant","content":"에"},"index":0,"inputLength":1265,"outputLength":1,"stopReason":null}
id:8dbd5681-c645-41b0-9be5-7015fc418154
event:token
data:{"message":{"role":"assistant","content":" 자동"},"index":0,"inputLength":1265,"outputLength":1,"sto

In [ ]:
user_info

'나는 주식에대해서 잘 몰라 그래서 미국 50대기업에 자동으로 투자하고 돈을 불려주는 상품을 추천받고 싶어'

In [ ]:
import json

answer = json.loads(output.replace('data:', ''))['message']['content']
answer

'미국 50대 기업에 자동으로 투자하고 돈을 불려주는 상품으로는 다음과 같은 ETF들이 있습니다. \n\n1. A472170 : 시장이 흔들려도 견고한 미국 빅테크 기업 및 AI라는 장기 신성장 동력을 얻은 미국 빅테크에 투자 가능한 ETF입니다. \n2. A474220 : 미국 시장을 대표하는 테크 TOP 10 기업에 투자 가능한 ETF입니다. 옵션 프리미엄으로 성장주에 월 배당을 더하고 있으며 TIGER +% 프리미엄 격주 배당 솔루션을 제공합니다.  \n3. A472160 : 미국의 성장을 가장 잘 반영하는 新 대표지수로서 견고한 미국 빅테크 기업 및 AI라는 신성장 동력을 얻은 미국 빅테크에 투자 가능한 ETF입니다.  \n\n위의 ETF들은 모두 미국 대형 기술주에 투자하는 상품으로, 자동으로 분산 투자가 가능하며 장기적으로 안정적인 수익을 기대할 수 있습니다. 다만, 투자 전에는 반드시 해당 상품의 운용 전략과 기초지수 등을 꼼꼼히 살펴보시고 자신의 투자 목적과 성향에 맞는 상품을 선택하시는 것이 중요합니다.'

## 4) Test

- Clova 자체 지식 활용 금지. 오롯이 DB에 근거하는지 테스트 필요.

In [ ]:
user_info_2 = "바이든 올해 몇 살이냐. 저번에 토론하는 거 보니까 참... 머쓱혔다."

In [ ]:
candidates_df_2 = find_top_k_sentences(user_info_2, etf_df_fixed, k=50)
candidates_df_2

,similarity,sentence,ticker,name
0,0.398480,[투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은...,A472170,TIGER미국테크TOP10채권혼합
1,0.397012,[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,429010,TIGER 미국나스닥넥스트100
2,0.395929,[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,441680,TIGER 미국나스닥100커버드콜(합성)
3,0.392770,[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,203780,TIGER 미국나스닥바이오
4,0.387661,[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,228810,TIGER 미디어컨텐츠
5,0.386676,[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,A474220,TIGER미국테크TOP10+10%프리미엄
6,0.386231,[주요보유현황] 채권명_만기_듀레이션_표면금리(%)_비중(%)_총듀레이션_BAC 5...,458260,TIGER 미국투자등급회사채액티브(H)
7,0.385551,[주요보유현황] 구분_종목명_구분_만기일_계약수량_(액면수량)_평가금액_(백만)_비...,137610,TIGER 농산물선물Enhanced(H)
8,0.382999,[투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은...,A472160,TIGER 미국테크TOP10INDXX(H)
9,0.380304,[거래정보] 상장거래소_KRX 유가증권시장_상장코드_245340_블룸버그티커(펀드)...,245340,TIGER 미국다우존스30


In [ ]:
candidates_2 = etf_df_fixed[etf_df_fixed['ticker'].isin(candidates_df_2['ticker'].unique())]

candidates_2 = candidates_2[candidates_2['risk'].isin(risk_dict[user_risk_level])]

if user_private_pension:
    candidates_2 = candidates_2[candidates_2['private'] == 'True']
if user_retire_pension:
    candidates_2 = candidates_2[candidates_2['retire'] == 'True']

filtered_tickers_2 = candidates_2['ticker'].values
filtered_tickers_2

array(['182490', '435420', '458260', '475630', '476550', 'A472170'],
      dtype=object)

In [ ]:
selected_df_2 = candidates_df_2[candidates_df_2['ticker'].isin(filtered_tickers_2)]
selected_df_2

,similarity,sentence,ticker,name
0,0.398480,[투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은...,A472170,TIGER미국테크TOP10채권혼합
6,0.386231,[주요보유현황] 채권명_만기_듀레이션_표면금리(%)_비중(%)_총듀레이션_BAC 5...,458260,TIGER 미국투자등급회사채액티브(H)
27,0.375433,[운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER...,476550,TIGER 미국30년국채프리미엄액티브(H)
29,0.374974,[거래정보] 상장거래소_KRX 유가증권시장_상장코드_182490_블룸버그티커(펀드)...,182490,TIGER 단기선진하이일드(합성 H)
35,0.371896,[거래정보] 상장거래소_KRX 유가증권시장_상장코드_458260_블룸버그티커(펀드)...,458260,TIGER 미국투자등급회사채액티브(H)
43,0.370295,[거래정보] 상장거래소_KRX 유가증권시장_상장코드_475630_블룸버그티커(펀드)...,475630,TIGER CD1년금리액티브(합성)
46,0.370017,[거래정보] 상장거래소_KRX 유가증권시장_상장코드_435420_블룸버그티커(펀드)...,435420,TIGER 미국나스닥100TR채권혼합Fn


In [ ]:
selected_sentences_2 = []

for idx, row in selected_df_2.iterrows():
    sent = f"<BOS> {row['ticker']}: {row['sentence']}"
    selected_sentences_2.append(sent)

texts_2 = ' <EOS> '.join(selected_sentences_2)
texts_2

'<BOS> A472170: [투자포인트] 1. 시장이흔들려도견고한미국빅테크기업_2. AI라는장기신성장동력을얻은미국빅테크_3. 퇴직연금내100%까지투자가능한ETF_ <EOS> <BOS> 458260: [주요보유현황] 채권명_만기_듀레이션_표면금리(%)_비중(%)_총듀레이션_BAC 5.015 07/22/33_2033.07.22_6.10_5.02_1.51_CSCO 4.85 02/26/29_2029.02.26_4.17_4.85_1.41_9.29_ABBV 3.2 11/21/29_2029.11.21_4.97_3.20_1.15_WFC 3.35 03/02/33_2033.03.02_6.50_3.35_1.06_HSBC 4.95 03/31/30_2030.03.31_5.02_4.95_1.02_KMI 5.2 06/01/33_2033.06.01_7.21_5.20_0.99_PM 4 7/8 02/13/29_2029.02.13_4.13_4.88_0.87_ENBCN 6.2 11/15/30_2030.11.15_5.37_6.20_0.84_JPM 4.912 07/25/33_2033.07.25_6.13_4.91_0.84_VZ 2.85 09/03/41_2041.09.03_12.72_2.85_0.74_주요보유현황전체종목비중_86.20_* 듀레이션(Duration)은해당종목(채권)의가중평균잔존만기를의미합니다. [업종비중] None [상위10종목] None <EOS> <BOS> 476550: [운용성과(%)] 구분_1M_3M_6M_1Y_3Y_YTD_설정이후_변동성_TIGER 미국30년국채프리미_2.45_-0.57_-_-_-_-_-0.24_6.62_KEDI US Treasury 30Y We_2.39_-1.45_-_-_-_-_0.84_7.35__ <EOS> <BOS> 182490: [거래정보] 상장거래소_KRX 유가증권시장_상장코드_182490_블룸버그티커(펀드)_182490 KS_블룸버그티커(기초지수)_IBXXSHY2_결제일_T+2_설정단위(CU)_50,000주_거래단위_1주_일거래량(최근3개월일평균)_3

In [ ]:
preset_text_2 = [{"role":"system",
                "content": ' '.join(["당신은 증권 전문가입니다.",
                                     "고객은 상장지수펀드 ETF에 관한 질문을 하고, 이는 <Query> 로 입력받습니다.",
                                     "ETF 정보를 담은 데이터베이스를 <DB> 로 입력받습니다.",
                                     "데이터베이스는 운용보고서를 발췌한 것으로, '<BOS> Ticker: [카테고리] 텍스트 <EOS>' 형태 문장으로 이뤄져있습니다."
                                     "반드시 데이터베이스 정보에 근거해 답변하십시오.",
                                     "답변할 때 '데이터베이스' 라는 단어는 사용하지 마십시오.",
                                     "데이터베이스에 없는 정보에 대해서는 '죄송합니다. 그 질문은 답변드릴 수 없습니다.' 라고만 답하십시오."])},
                  {"role":"user","content":f"<Query>: {user_info_2} </Query> <DB> {texts_2} </DB>"}]

request_data_2 = {
    'messages': preset_text_2,
    'topP': 0.8,
    'topK': 0,
    'maxTokens': 2048,
    'temperature': 0.5, 'repeatPenalty': 5.0,
    'stopBefore': [],
    'includeAiFilters': True,
    'seed': 42
}

output_2 = clova_chat.execute(request_data_2)

id:02193c15-0fe6-4d60-897b-350b4826cc5b
event:token
data:{"message":{"role":"assistant","content":"죄송합니다"},"index":0,"inputLength":1161,"outputLength":1,"stopReason":null}
id:099fa677-5a44-4c2e-a19e-80b1c86bf8bc
event:token
data:{"message":{"role":"assistant","content":"."},"index":0,"inputLength":1161,"outputLength":1,"stopReason":null}
id:4162695b-284c-4069-833d-37952acaa764
event:token
data:{"message":{"role":"assistant","content":" 그"},"index":0,"inputLength":1161,"outputLength":1,"stopReason":null}
id:81a51af9-500b-41d6-80c3-e6e2d622ef23
event:token
data:{"message":{"role":"assistant","content":" 질문"},"index":0,"inputLength":1161,"outputLength":1,"stopReason":null}
id:4439a476-8af4-43dc-9e6d-2b76a39652fc
event:token
data:{"message":{"role":"assistant","content":"은"},"index":0,"inputLength":1161,"outputLength":1,"stopReason":null}
id:48b9d72e-ee61-4844-bf0e-928c69fc8800
event:token
data:{"message":{"role":"assistant","content":" 답변"},"index":0,"inputLength":1161,"outputLength":1,"s

In [ ]:
user_info_2 = "바이든 올해 몇 살이냐. 저번에 토론하는 거 보니까 참... 머쓱혔다."

In [ ]:
answer_2 = json.loads(output_2.replace('data:', ''))['message']['content']
answer_2

'죄송합니다. 그 질문은 답변드릴 수 없습니다.'